# **Chapter 47: Project 4 - Streaming/Media App**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Architect a media streaming application with proper lifecycle management for video/audio playback
- Implement video playback using `video_player` and `chewie` with custom controls and error handling
- Configure background audio playback with lock screen controls, metadata, and remote command center integration
- Implement DRM (Digital Rights Management) using Widevine (Android) and FairPlay (iOS) for content protection
- Support adaptive bitrate streaming (HLS for iOS, DASH for Android) for optimal quality across network conditions
- Integrate casting protocols (Chromecast via Google Cast SDK, AirPlay via AVRoutePickerView)
- Implement Picture-in-Picture (PiP) mode for multitasking during playback
- Manage media downloads for offline viewing with encrypted storage

---

## **Prerequisites**

- Completed Chapter 46: Project 3 - Productivity/Task Management App (complex state management)
- Completed Chapter 38: Plugin Development (understanding of platform channels for DRM/casting)
- Understanding of video codecs (H.264, H.265/HEVC) and container formats (MP4, MKV, HLS, DASH)
- Familiarity with media streaming concepts (bitrate, resolution, buffering)
- Apple Developer account (for FairPlay DRM testing on iOS)
- Google Cast SDK registration (for Chromecast integration)

---

## **47.1 Architecture Overview**

Media apps require careful lifecycle management due to the heavy resource usage of video decoders and the complexity of background audio sessions.

### **Architecture Pattern: BLoC with Player Service**

```
lib/
├── core/
│   └── player/
│       ├── media_player_service.dart      # Singleton managing native players
│       ├── player_bloc.dart               # State: playing, paused, buffering, error
│       └── player_controls.dart           # UI components
├── features/
│   ├── video_player/
│   │   ├── presentation/
│   │   │   ├── video_screen.dart        # Full-screen video UI
│   │   │   └── pip_manager.dart         # Picture-in-Picture
│   │   └── logic/
│   │       └── video_bloc.dart
│   ├── audio_player/
│   │   ├── service/
│   │   │   └── audio_handler.dart       # audio_service implementation
│   │   └── presentation/
│   │       └── mini_player.dart          # Persistent bottom player
│   └── downloads/
│       ├── data/
│       │   └── download_manager.dart      # DRM-aware downloads
│       └── encryption/
└── platform/
    └── drm/
        ├── widevine_manager.dart         # Android DRM
        └── fairplay_manager.dart         # iOS DRM
```

**Explanation:**

- **Media Player Service**: A singleton that wraps the native `VideoPlayerController` and `AudioPlayer`. Ensures only one media item plays at a time and manages system audio focus.
- **Audio Handler**: Required for background audio. Implements `BaseAudioHandler` from `audio_service` to provide metadata to the system (lock screen, CarPlay, Android Auto).
- **DRM Managers**: Platform-specific implementations using MethodChannels to access native DRM APIs (ExoPlayer for Android, AVPlayer for iOS).

---

## **47.2 Video Playback Implementation**

Using `video_player` for low-level control and `chewie` for UI, or building custom controls for branded experiences.

### **Basic Video Player Setup**

```dart
// lib/core/player/media_player_service.dart
import 'package:video_player/video_player.dart';
import 'package:flutter/services.dart';

@singleton
class MediaPlayerService {
  VideoPlayerController? _videoController;
  final StreamController<PlayerState> _stateController = StreamController.broadcast();
  
  Stream<PlayerState> get stateStream => _stateController.stream;
  VideoPlayerController? get controller => _videoController;

  Future<void> initializeVideo({
    required String url,
    Map<String, String>? headers,
    bool autoPlay = false,
  }) async {
    // Dispose previous controller to free resources
    await _videoController?.dispose();
    
    // Create new controller based on URL type
    if (url.contains('.m3u8')) {
      // HLS streaming
      _videoController = VideoPlayerController.network(
        url,
        formatHint: VideoFormat.hls,
        httpHeaders: headers ?? {},
      );
    } else if (url.contains('.mpd')) {
      // DASH streaming (Android only)
      _videoController = VideoPlayerController.network(
        url,
        formatHint: VideoFormat.dash,
        httpHeaders: headers ?? {},
      );
    } else {
      // Progressive download (MP4)
      _videoController = VideoPlayerController.network(url);
    }

    // Initialize and setup listeners
    await _videoController!.initialize();
    
    _videoController!.addListener(_onPlayerStateChanged);
    
    if (autoPlay) {
      await play();
    }
    
    _stateController.add(PlayerState.ready);
  }

  void _onPlayerStateChanged() {
    if (_videoController == null) return;
    
    final value = _videoController!.value;
    
    if (value.isBuffering) {
      _stateController.add(PlayerState.buffering);
    } else if (value.isPlaying) {
      _stateController.add(PlayerState.playing);
    } else if (value.position >= value.duration) {
      _stateController.add(PlayerState.completed);
    } else {
      _stateController.add(PlayerState.paused);
    }
    
    // Handle errors
    if (value.hasError) {
      _stateController.add(PlayerState.error(value.errorDescription));
    }
  }

  Future<void> play() async {
    await _videoController?.play();
    // Request audio focus
    await _setAudioFocus();
  }

  Future<void> pause() async {
    await _videoController?.pause();
  }

  Future<void> seekTo(Duration position) async {
    await _videoController?.seekTo(position);
  }

  Future<void> setPlaybackSpeed(double speed) async {
    await _videoController?.setPlaybackSpeed(speed);
  }

  Future<void> dispose() async {
    await _videoController?.dispose();
    _videoController = null;
    await _stateController.close();
  }

  Future<void> _setAudioFocus() async {
    // Configure audio session for video playback
    final session = await AudioSession.instance;
    await session.configure(const AudioSessionConfiguration(
      avAudioSessionCategory: AVAudioSessionCategory.playback,
      avAudioSessionMode: AVAudioSessionMode.moviePlayback,
    ));
    await session.setActive(true);
  }
}

enum PlayerState {
  idle,
  ready,
  playing,
  paused,
  buffering,
  completed,
  error(String? message);
}
```

**Explanation:**

- **Format Hints**: Explicitly setting `VideoFormat.hls` or `VideoFormat.dash` helps the underlying native player (ExoPlayer on Android, AVPlayer on iOS) choose the correct demuxer.
- **Resource Management**: Always `dispose()` the previous controller before creating a new one. Video decoders are hardware-accelerated and consume significant GPU/CPU resources.
- **Audio Session**: Configures the system audio focus. Prevents the video from being silent when the device is on silent mode (critical for user experience).

### **Custom Video Controls**

```dart
// lib/features/video_player/presentation/widgets/custom_controls.dart
class CustomVideoControls extends StatelessWidget {
  final VideoPlayerController controller;
  final VoidCallback onCast;
  final VoidCallback onEnterPip;

  const CustomVideoControls({
    super.key,
    required this.controller,
    required this.onCast,
    required this.onEnterPip,
  });

  @override
  Widget build(BuildContext context) {
    return ValueListenableBuilder<VideoPlayerValue>(
      valueListenable: controller,
      builder: (context, value, child) {
        return Stack(
          children: [
            // Gradient overlay for visibility
            Positioned(
              bottom: 0,
              left: 0,
              right: 0,
              child: Container(
                height: 80,
                decoration: BoxDecoration(
                  gradient: LinearGradient(
                    begin: Alignment.topCenter,
                    end: Alignment.bottomCenter,
                    colors: [
                      Colors.transparent,
                      Colors.black.withOpacity(0.7),
                    ],
                  ),
                ),
              ),
            ),
            
            // Controls
            Positioned(
              bottom: 0,
              left: 0,
              right: 0,
              child: Row(
                children: [
                  // Play/Pause
                  IconButton(
                    icon: Icon(
                      value.isPlaying ? Icons.pause : Icons.play_arrow,
                      color: Colors.white,
                    ),
                    onPressed: () {
                      value.isPlaying ? controller.pause() : controller.play();
                    },
                  ),
                  
                  // Progress bar
                  Expanded(
                    child: VideoProgressIndicator(
                      controller,
                      allowScrubbing: true,
                      colors: const VideoProgressColors(
                        playedColor: Colors.red,
                        bufferedColor: Colors.grey,
                        backgroundColor: Colors.white24,
                      ),
                    ),
                  ),
                  
                  // Duration
                  Text(
                    '${_formatDuration(value.position)} / ${_formatDuration(value.duration)}',
                    style: const TextStyle(color: Colors.white),
                  ),
                  
                  // Cast button
                  IconButton(
                    icon: const Icon(Icons.cast, color: Colors.white),
                    onPressed: onCast,
                  ),
                  
                  // PiP button (iPad/Android tablets)
                  if (Platform.isIOS || Platform.isAndroid)
                    IconButton(
                      icon: const Icon(Icons.picture_in_picture, color: Colors.white),
                      onPressed: onEnterPip,
                    ),
                  
                  // Fullscreen toggle
                  IconButton(
                    icon: const Icon(Icons.fullscreen, color: Colors.white),
                    onPressed: () {
                      // Toggle orientation or enter immersive mode
                      _toggleFullscreen(context);
                    },
                  ),
                ],
              ),
            ),
            
            // Buffering indicator
            if (value.isBuffering)
              const Center(
                child: CircularProgressIndicator(),
              ),
          ],
        );
      },
    );
  }

  String _formatDuration(Duration duration) {
    String twoDigits(int n) => n.toString().padLeft(2, '0');
    final hours = twoDigits(duration.inHours);
    final minutes = twoDigits(duration.inMinutes.remainder(60));
    final seconds = twoDigits(duration.inSeconds.remainder(60));
    
    return duration.inHours > 0 
        ? '$hours:$minutes:$seconds' 
        : '$minutes:$seconds';
  }

  void _toggleFullscreen(BuildContext context) {
    if (MediaQuery.of(context).orientation == Orientation.portrait) {
      SystemChrome.setPreferredOrientations([
        DeviceOrientation.landscapeLeft,
        DeviceOrientation.landscapeRight,
      ]);
      SystemChrome.setEnabledSystemUIMode(SystemUiMode.immersiveSticky);
    } else {
      SystemChrome.setPreferredOrientations([DeviceOrientation.portraitUp]);
      SystemChrome.setEnabledSystemUIMode(SystemUiMode.edgeToEdge);
    }
  }
}
```

**Explanation:**

- **ValueListenableBuilder**: Efficiently rebuilds only when the video player state changes (position, buffering status, etc.).
- **Immersive Mode**: Hides system UI (status bar, navigation) during fullscreen video playback.
- **Scrubbing**: `VideoProgressIndicator` with `allowScrubbing: true` enables drag-to-seek functionality.
- **Orientation Management**: Programmatically forcing landscape for fullscreen video is standard UX for media apps.

---

## **47.3 Background Audio Playback**

For music or podcast features, implement background audio with lock screen controls using `audio_service` and `just_audio`.

```dart
// lib/features/audio_player/service/audio_handler.dart
import 'package:audio_service/audio_service.dart';
import 'package:just_audio/just_audio.dart';

class AudioPlayerHandler extends BaseAudioHandler with QueueHandler, SeekHandler {
  final _player = AudioPlayer();
  final _playlist = ConcatenatingAudioSource(children: []);

  AudioPlayerHandler() {
    _init();
  }

  Future<void> _init() async {
    // Load empty playlist
    await _player.setAudioSource(_playlist);
    
    // Propagate player events to audio_service
    _player.playbackEventStream.map(_transformEvent).pipe(playbackState);
    
    // Handle media item changes
    _player.currentIndexStream.listen((index) {
      if (index != null && index < queue.value.length) {
        mediaItem.add(queue.value[index]);
      }
    });
  }

  PlaybackState _transformEvent(PlaybackEvent event) {
    return PlaybackState(
      controls: [
        MediaControl.skipToPrevious,
        if (_player.playing) MediaControl.pause else MediaControl.play,
        MediaControl.stop,
        MediaControl.skipToNext,
      ],
      systemActions: const {
        MediaAction.seek,
        MediaAction.seekForward,
        MediaAction.seekBackward,
      },
      androidCompactActionIndices: const [0, 1, 3],
      processingState: const {
        ProcessingState.idle: AudioProcessingState.idle,
        ProcessingState.loading: AudioProcessingState.loading,
        ProcessingState.buffering: AudioProcessingState.buffering,
        ProcessingState.ready: AudioProcessingState.ready,
        ProcessingState.completed: AudioProcessingState.completed,
      }[_player.processingState]!,
      playing: _player.playing,
      updatePosition: _player.position,
      bufferedPosition: _player.bufferedPosition,
      speed: _player.speed,
      queueIndex: _player.currentIndex,
    );
  }

  Future<void> addQueueItem(MediaItem mediaItem) async {
    final audioSource = AudioSource.uri(
      Uri.parse(mediaItem.extras!['url']),
      tag: mediaItem,
    );
    await _playlist.add(audioSource);
    queue.add([...queue.value, mediaItem]);
  }

  @override
  Future<void> play() => _player.play();

  @override
  Future<void> pause() => _player.pause();

  @override
  Future<void> seek(Duration position) => _player.seek(position);

  @override
  Future<void> skipToQueueItem(int index) async {
    await _player.seek(Duration.zero, index: index);
    play();
  }

  @override
  Future<void> skipToNext() => _player.seekToNext();

  @override
  Future<void> skipToPrevious() => _player.seekToPrevious();

  @override
  Future<void> stop() async {
    await _player.stop();
    await super.stop();
  }
}

// Initialization in main.dart
void main() async {
  WidgetsFlutterBinding.ensureInitialized();
  
  await AudioService.init(
    builder: () => AudioPlayerHandler(),
    config: const AudioServiceConfig(
      androidNotificationChannelId: 'com.example.audio.channel',
      androidNotificationChannelName: 'Audio Playback',
      androidNotificationOngoing: true,
      androidStopForegroundOnPause: false,
    ),
  );
  
  runApp(MyApp());
}
```

**Explanation:**

- **BaseAudioHandler**: Abstract base class from `audio_service`. Implementing `QueueHandler` and `SeekHandler` mixins provides queue and seeking capabilities.
- **AudioSource**: `just_audio` supports various sources: `AudioSource.uri` for network files, `AudioSource.file` for local downloads, `HlsAudioSource` for HLS streams.
- **Notification Channel**: `androidNotificationChannelId` creates a persistent notification on Android that keeps the service alive in the background.
- **MediaItem**: Standard metadata format (title, artist, album art, duration) displayed on lock screen and CarPlay/Android Auto.

---

## **47.4 DRM Implementation**

Protecting premium content requires platform-specific DRM implementations.

### **Android Widevine Implementation**

```dart
// lib/platform/drm/widevine_manager.dart
import 'package:flutter/services.dart';

class WidevineManager {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/drm');

  /// Initialize DRM session for a video
  static Future<DrmSession> initializeWidevine({
    required String licenseUrl,
    required Map<String, String> headers,
  }) async {
    try {
      final result = await _channel.invokeMethod<Map>('initializeWidevine', {
        'licenseUrl': licenseUrl,
        'headers': headers,
      });
      
      return DrmSession(
        sessionId: result!['sessionId'],
        licenseAcquired: result['licenseAcquired'],
      );
    } catch (e) {
      throw DrmException('Failed to initialize Widevine: $e');
    }
  }

  /// Release DRM session
  static Future<void> releaseSession(String sessionId) async {
    await _channel.invokeMethod('releaseDrmSession', {
      'sessionId': sessionId,
    });
  }
}

class DrmSession {
  final String sessionId;
  final bool licenseAcquired;

  DrmSession({
    required this.sessionId,
    required this.licenseAcquired,
  });
}

// Android native implementation (Kotlin)
/*
class DrmPlugin : FlutterPlugin, MethodCallHandler {
    private lateinit var context: Context
    
    override fun onMethodCall(call: MethodCall, result: Result) {
        when (call.method) {
            "initializeWidevine" -> {
                val licenseUrl = call.argument<String>("licenseUrl")!!
                val headers = call.argument<Map<String, String>>("headers")!!
                
                try {
                    val mediaDrm = MediaDrm(WIDEVINE_UUID)
                    val sessionId = mediaDrm.openSession()
                    
                    // Execute key request
                    val keyRequest = mediaDrm.getKeyRequest(
                        sessionId,
                        initData,
                        "cenc",
                        MediaDrm.KEY_TYPE_STREAMING,
                        headers
                    )
                    
                    // Network request to license server
                    val license = fetchLicense(licenseUrl, keyRequest.data, headers)
                    mediaDrm.provideKeyResponse(sessionId, license)
                    
                    result.success(mapOf(
                        "sessionId" to sessionId.toBase64(),
                        "licenseAcquired" to true
                    ))
                } catch (e: Exception) {
                    result.error("DRM_ERROR", e.message, null)
                }
            }
        }
    }
}
*/
```

**Explanation:**

- **Widevine**: Google's DRM system on Android. `MediaDrm` class manages the cryptographic session.
- **License Acquisition**: The player requests a license from the DRM server. The license contains decryption keys for the content.
- **Security Levels**: Widevine has L1 (hardware decryption), L2 (software decryption), and L3 (software only, least secure). Check `securityLevel` to ensure device compatibility.
- **MethodChannel**: Required because Flutter's `video_player` doesn't expose DRM APIs directly. You must implement custom platform code or use plugins like `better_player` or `flutter_vlc_player` that support DRM.

### **iOS FairPlay Implementation**

```swift
// iOS native implementation
import Flutter
import AVFoundation

public class DrmPlugin: NSObject, FlutterPlugin {
    public static func register(with registrar: FlutterPluginRegistrar) {
        let channel = FlutterMethodChannel(name: "com.example.app/drm", binaryMessenger: registrar.messenger())
        let instance = DrmPlugin()
        registrar.addMethodCallDelegate(instance, channel: channel)
    }
    
    public func handle(_ call: FlutterMethodCall, result: @escaping FlutterResult) {
        if call.method == "initializeFairPlay" {
            guard let args = call.arguments as? [String: Any],
                  let certificateUrl = args["certificateUrl"] as? String,
                  let licenseUrl = args["licenseUrl"] as? String else {
                result(FlutterError(code: "INVALID_ARGS", message: nil, details: nil))
                return
            }
            
            // FairPlay requires application certificate
            guard let certData = try? Data(contentsOf: URL(string: certificateUrl)!) else {
                result(FlutterError(code: "CERT_ERROR", message: "Failed to load certificate", details: nil))
                return
            }
            
            // Store certificate and license URL for AVContentKeySession
            FairPlaySessionManager.shared.setup(
                certificate: certData,
                licenseUrl: licenseUrl
            )
            
            result(["status": "initialized"])
        }
    }
}

// Usage with AVPlayer
class FairPlayResourceLoaderDelegate: NSObject, AVAssetResourceLoaderDelegate {
    func resourceLoader(_ resourceLoader: AVAssetResourceLoader, 
                       shouldWaitForLoadingOfRequestedResource loadingRequest: AVAssetResourceLoadingRequest) -> Bool {
        // Handle FairPlay key requests here
        // 1. Extract SPC (Server Playback Context)
        // 2. Send to license server
        // 3. Provide CKC (Content Key Context) to player
        return true
    }
}
```

**Explanation:**

- **FairPlay**: Apple's DRM system. Requires an "Application Certificate" from Apple, signed with your developer certificate.
- **AVContentKeySession**: iOS API for managing DRM keys. The resource loader delegate intercepts key requests from the player.
- **SPC/CKC**: FairPlay uses a challenge-response protocol. The player generates an SPC (challenge), sends it to the license server, and receives a CKC (response) containing the decryption key.

---

## **47.5 Adaptive Streaming (HLS/DASH)**

Implementing adaptive bitrate streaming to adjust quality based on network conditions.

```dart
// lib/core/player/adaptive_streaming.dart
class AdaptiveStreamingConfig {
  /// For HLS (iOS native, Android via ExoPlayer)
  static VideoPlayerController createHlsPlayer(String url) {
    return VideoPlayerController.network(
      url,
      formatHint: VideoFormat.hls,
      // On iOS, AVPlayer automatically handles variant selection
      // On Android, ExoPlayer uses adaptive track selection
    );
  }

  /// For DASH (Android primarily, iOS via third-party)
  static VideoPlayerController createDashPlayer(String url) {
    if (Platform.isAndroid) {
      return VideoPlayerController.network(
        url,
        formatHint: VideoFormat.dash,
      );
    } else {
      // iOS doesn't support DASH natively, convert to HLS or use plugin
      throw UnsupportedError('DASH not supported on iOS');
    }
  }
}

// Quality selection (manual override)
class QualitySelector extends StatelessWidget {
  final VideoPlayerController controller;
  final List<VideoQuality> availableQualities;

  const QualitySelector({
    super.key,
    required this.controller,
    required this.availableQualities,
  });

  @override
  Widget build(BuildContext context) {
    return PopupMenuButton<VideoQuality>(
      icon: const Icon(Icons.settings),
      onSelected: (quality) => _setQuality(quality),
      itemBuilder: (context) {
        return [
          const PopupMenuItem(
            value: null,
            child: Text('Auto'),
          ),
          ...availableQualities.map((q) {
            return PopupMenuItem(
              value: q,
              child: Text('${q.height}p'),
            );
          }),
        ];
      },
    );
  }

  void _setQuality(VideoQuality? quality) {
    // Note: video_player doesn't expose track selection APIs directly
    // Requires native implementation or plugins like better_player
    if (quality == null) {
      // Enable ABR (Adaptive Bitrate)
      _enableAdaptiveBitrate();
    } else {
      // Force specific track
      _forceQuality(quality);
    }
  }

  void _enableAdaptiveBitrate() {
    // Platform channel to ExoPlayer/AVPlayer
    // ExoPlayer: trackSelector.setParameters(
    //   trackSelector.buildUponParameters().setMaxVideoSizeSd()
    // )
  }

  void _forceQuality(VideoQuality quality) {
    // Platform channel to select specific track
  }
}

class VideoQuality {
  final int height; // 720, 1080, etc.
  final int bitrate;
  final String codec;

  VideoQuality({
    required this.height,
    required this.bitrate,
    required this.codec,
  });
}
```

**Explanation:**

- **HLS (HTTP Live Streaming)**: Apple's protocol. Uses `.m3u8` playlist files containing multiple variant streams (360p, 720p, 1080p). The player automatically switches based on bandwidth.
- **DASH (Dynamic Adaptive Streaming)**: MPEG standard, uses `.mpd` manifest files. Better supported on Android via ExoPlayer.
- **Track Selection**: Standard `video_player` doesn't expose APIs to manually select quality. For advanced control, use `better_player` or implement platform channels to ExoPlayer's `DefaultTrackSelector`.

---

## **47.6 Casting Support (Chromecast/AirPlay)**

### **Chromecast Integration**

```dart
// lib/features/cast/chromecast_manager.dart
import 'package:google_cast/google_cast.dart';

class ChromecastManager {
  final GoogleCastContext _castContext = GoogleCastContext.instance;
  final StreamController<CastState> _stateController = StreamController.broadcast();

  Stream<CastState> get stateStream => _stateController.stream;

  Future<void> initialize() async {
    await _castContext.setOptions(
      GoogleCastOptions(
        receiverApplicationId: GoogleCastContext.defaultMediaReceiver,
      ),
    );
    
    // Listen for session state changes
    GoogleCastSessionManager.instance.currentSession?.stateStream.listen((state) {
      _stateController.add(state);
    });
  }

  Future<void> castVideo(String url, MediaMetadata metadata) async {
    final session = GoogleCastSessionManager.instance.currentSession;
    if (session == null || !session.isConnected) {
      // Show device picker
      await GoogleCastContext.instance.showCastDialog();
      return;
    }

    final mediaInfo = GoogleCastMediaInformation(
      contentId: url,
      contentType: 'video/mp4',
      streamType: GoogleCastStreamType.buffered,
      metadata: GoogleCastMediaMetadata(
        type: GoogleCastMediaMetadataType.movie,
        title: metadata.title,
        subtitle: metadata.subtitle,
        images: [
          GoogleCastImage(
            url: metadata.posterUrl,
            width: 480,
            height: 720,
          ),
        ],
      ),
    );

    final request = GoogleCastMediaLoadRequest(mediaInfo);
    await session.remoteMediaClient?.load(request);
  }

  Future<void> stopCasting() async {
    await GoogleCastSessionManager.instance.endSession();
  }
}
```

**Explanation:**

- **Google Cast SDK**: Requires registration at [Google Cast SDK Developer Console](https://developers.google.com/cast/docs/registration) to get an App ID.
- **Sender App**: Flutter app acts as the "sender". It discovers Cast devices on the network and controls playback.
- **Receiver**: The Chromecast runs a receiver app (usually the Default Media Receiver for video, or a custom Styled Receiver).
- **MediaInformation**: Metadata sent to the Chromecast for display on the TV (title, poster art, progress).

### **AirPlay Integration**

```dart
// lib/features/cast/airplay_button.dart
import 'package:flutter/services.dart';

class AirPlayButton extends StatelessWidget {
  static const platform = MethodChannel('com.example.app/airplay');

  const AirPlayButton({super.key});

  @override
  Widget build(BuildContext context) {
    if (!Platform.isIOS) return const SizedBox();

    return InkWell(
      onTap: _showAirPlayPicker,
      child: Container(
        width: 44,
        height: 44,
        child: const UiKitView(
          viewType: 'AirPlayRoutePickerView',
          creationParams: <String, dynamic>{
            'activeTintColor': '#FFFFFF',
            'tintColor': '#FFFFFF',
          },
          creationParamsCodec: StandardMessageCodec(),
        ),
      ),
    );
  }

  void _showAirPlayPicker() async {
    try {
      await platform.invokeMethod('showAirPlayPicker');
    } catch (e) {
      print('Failed to show AirPlay picker: $e');
    }
  }
}

// iOS native implementation
/*
import AVKit

class AirPlayView: NSObject, FlutterPlatformView {
    func view() -> UIView {
        let routePickerView = AVRoutePickerView()
        routePickerView.tintColor = .white
        routePickerView.activeTintColor = .white
        return routePickerView
    }
}
*/
```

**Explanation:**

- **AVRoutePickerView**: iOS native button that automatically handles AirPlay device discovery and connection. The system handles the UI (picker sheet).
- **PlatformView**: Embeds the native iOS view into the Flutter widget tree using `UiKitView`.
- **AirPlay**: Automatically routes audio/video to Apple TV or AirPlay-compatible speakers. For video, the system handles the stream URL; your app just needs to initiate the connection.

---

## **47.7 Picture-in-Picture (PiP)**

Allowing video to play in a floating window while the user navigates other apps.

```dart
// lib/features/video_player/presentation/pip_manager.dart
import 'package:flutter/services.dart';

class PipManager {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/pip');

  /// Check if PiP is supported on this device
  static Future<bool> isSupported() async {
    if (!Platform.isIOS && !Platform.isAndroid) return false;
    
    try {
      return await _channel.invokeMethod<bool>('isPipSupported') ?? false;
    } catch (e) {
      return false;
    }
  }

  /// Enter PiP mode
  static Future<void> enterPipMode() async {
    try {
      await _channel.invokeMethod('enterPipMode');
    } catch (e) {
      throw PipException('Failed to enter PiP: $e');
    }
  }

  /// Configure PiP aspect ratio (iOS only)
  static Future<void> configureAspectRatio(double width, double height) async {
    if (Platform.isIOS) {
      await _channel.invokeMethod('configurePipAspectRatio', {
        'width': width,
        'height': height,
      });
    }
  }
}

// Android native implementation
/*
class PipPlugin : FlutterPlugin, MethodCallHandler, ActivityAware {
    private lateinit var activity: Activity
    
    override fun onMethodCall(call: MethodCall, result: Result) {
        when (call.method) {
            "enterPipMode" -> {
                if (Build.VERSION.SDK_INT >= Build.VERSION_CODES.O) {
                    val params = PictureInPictureParams.Builder()
                        .setAspectRatio(Rational(16, 9))
                        .build()
                    activity.enterPictureInPictureMode(params)
                    result.success(null)
                } else {
                    result.error("UNSUPPORTED", "PiP requires Android 8.0+", null)
                }
            }
        }
    }
}
*/

// iOS native implementation
/*
import AVKit
import Flutter

class PipPlugin: NSObject, FlutterPlugin {
    var pipController: AVPictureInPictureController?
    
    func handle(_ call: FlutterMethodCall, result: @escaping FlutterResult) {
        if call.method == "enterPipMode" {
            guard let playerLayer = videoPlayerView.playerLayer else {
                result(FlutterError(code: "NO_PLAYER", message: nil, details: nil))
                return
            }
            
            pipController = AVPictureInPictureController(playerLayer: playerLayer)
            pipController?.startPictureInPicture()
            result(nil)
        }
    }
}
*/
```

**Explanation:**

- **Android PiP**: Uses `PictureInPictureParams` to configure aspect ratio. Requires `android:supportsPictureInPicture="true"` in AndroidManifest.xml activity declaration.
- **iOS PiP**: Uses `AVPictureInPictureController` which requires the app to have the "Background Mode: Audio, AirPlay, and Picture in Picture" capability enabled in Xcode.
- **Aspect Ratio**: Should match the video content (16:9 for landscape, 9:16 for portrait) to avoid black bars.

---

## **47.8 Offline Downloads with Encryption**

Allowing users to download content for offline viewing while protecting against unauthorized copying.

```dart
// lib/features/downloads/data/download_manager.dart
import 'package:flutter_downloader/flutter_downloader.dart';
import 'package:encrypt/encrypt.dart';

class EncryptedDownloadManager {
  final _downloadQueue = <DownloadTask>[];
  
  Future<String> downloadVideo({
    required String url,
    required String videoId,
    required String licenseKey, // For decryption
  }) async {
    // Create encrypted filename
    final fileName = '$videoId.enc';
    final savePath = await _getDownloadPath();
    final fullPath = '$savePath/$fileName';
    
    // Start download
    final taskId = await FlutterDownloader.enqueue(
      url: url,
      savedDir: savePath,
      fileName: fileName,
      showNotification: true,
      openFileFromNotification: false,
      saveInPublicStorage: false,
    );
    
    // Store metadata in local DB
    await _storeDownloadMetadata(
      taskId: taskId!,
      videoId: videoId,
      filePath: fullPath,
      encryptionKey: licenseKey,
    );
    
    return taskId;
  }

  Future<String> getDecryptedFilePath(String videoId) async {
    final metadata = await _getDownloadMetadata(videoId);
    final encryptedFile = File(metadata.filePath);
    
    if (!await encryptedFile.exists()) {
      throw DownloadException('File not found');
    }
    
    // Decrypt to temporary file for playback
    final tempDir = await getTemporaryDirectory();
    final tempPath = '${tempDir.path}/$videoId.mp4';
    
    final key = Key.fromBase64(metadata.encryptionKey);
    final iv = IV.fromLength(16);
    final encrypter = Encrypter(AES(key, mode: AESMode.cbc));
    
    final encryptedBytes = await encryptedFile.readAsBytes();
    final decrypted = encrypter.decryptBytes(Encrypted(encryptedBytes), iv: iv);
    
    await File(tempPath).writeAsBytes(decrypted);
    
    // Schedule cleanup of temp file after playback
    _scheduleTempCleanup(tempPath);
    
    return tempPath;
  }

  Future<String> _getDownloadPath() async {
    final dir = await getApplicationDocumentsDirectory();
    final downloadDir = Directory('${dir.path}/downloads');
    if (!await downloadDir.exists()) {
      await downloadDir.create(recursive: true);
    }
    return downloadDir.path;
  }
}
```

**Explanation:**

- **flutter_downloader**: Handles background downloads with progress notifications. Downloads to app-private storage.
- **AES Encryption**: Encrypts the downloaded file using a key derived from the DRM license or user credentials. This prevents users from extracting the raw MP4 from the app sandbox.
- **Temporary Decryption**: Decrypts to a temporary file only during playback. The temp file is deleted after use to prevent copying.
- **Secure Storage**: The encryption key should be stored in Android Keystore/iOS Keychain, not in the database.

---

## **Chapter Summary**

In this chapter, we built a comprehensive streaming/media application:

### **Key Takeaways:**

1. **Video Lifecycle**: Always dispose controllers to free hardware decoders. Use `ValueListenableBuilder` for efficient UI updates.
2. **Background Audio**: Implement `AudioHandler` from `audio_service` to provide lock screen controls and enable background playback.
3. **DRM Protection**: Use Widevine (Android) and FairPlay (iOS) via platform channels. Requires native implementation as Flutter's video_player doesn't expose DRM APIs.
4. **Adaptive Streaming**: HLS for iOS, DASH for Android. The player automatically selects quality based on bandwidth.
5. **Casting**: Chromecast requires the Google Cast SDK and a receiver app. AirPlay uses `AVRoutePickerView` for native device selection.
6. **Picture-in-Picture**: Requires native implementation using `PictureInPictureParams` (Android) and `AVPictureInPictureController` (iOS).
7. **Offline Downloads**: Download to encrypted storage. Decrypt temporarily for playback to prevent unauthorized sharing.

### **Performance Considerations:**
- ✅ Use `cached_network_image` for video thumbnails
- ✅ Preload video metadata but don't initialize multiple players simultaneously
- ✅ Release audio focus when not playing
- ✅ Use `androidStopForegroundOnPause: false` to keep notification active during buffering

---

## **Next Steps**

In the next chapter, **Chapter 48: Performance Optimization**, we will explore advanced techniques for ensuring 60fps performance in complex Flutter applications. You'll learn how to use DevTools for profiling, implement custom painters for optimized rendering, manage memory leaks, and optimize shader compilation for first-frame rendering.

---

**End of Chapter 47**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='46. task_management_app.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../13. Advanced_topics_and_reference/48. performance_optimization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
